Script for fish quantitative metabarcoding project<br>
Pieternella Luttikhuizen, 2018

In [ ]:
# only need to run this once
import sys
!{sys.executable} -m pip install matplotlib

In [ ]:
import csv
import matplotlib.pyplot as plt
import matplotlib
import numpy
import math

The code below is meant to make a graph of the numbers of fish caught per species.

In [ ]:
# data from NIOZ fike from 2017 to analyse quantities of fish caught per species
with open('Species2017Week_fish_only.csv') as f:
    reader = csv.reader(f)
    headers = reader.__next__()
    print('headers:', headers)
    species = []
    total_number = []
    for row in f:
        row = row.strip('\n')
        row = row.split(',')
        species.append(row[0])
        total_number.append(int(row[33]))
log10number = [math.log10(x) for x in total_number]
print(species,total_number,log10number)

In [ ]:
y_pos = numpy.arange(len(species))

# Create bars
plt.bar(y_pos, log10number)
 
# Create names on the x-axis
plt.xticks(y_pos, species, rotation=90, size = 7)
plt.ylabel('log10(number of fish)')

# Custom the subplot layout
#plt.subplots_adjust(bottom=0.4, top=0.99)

# Add horizontal line
plt.hlines(math.log10(30), 0, 45, linestyles='dotted')
plt.text(40, 1.6, 'n=30', ha='left', va='center')

# Show graphic
plt.savefig('fish_numbers_2017.png', bbox_inches='tight')
plt.show()

In [ ]:
print(y_pos)

The code below is used to <br>
1) obtain the sequence data per species from boldsystems.org using their API<br>
2) make a single fasta file containing all COI data for all species in the folder COI_data_boldsystems.

In [ ]:
# install package requests if you do not yet have it installed
# only need to run this once
import sys
!{sys.executable} -m pip install requests

In [ ]:
# read the list of species we are looking for from infile
fish_species_names = []
with open('COI_data_boldsystems/fish_species_names.txt') as f:
    for line in f:
        fish_species_names.append(line.strip())
f.close()

In [ ]:
# 1 obtain data via API
import requests
for fish_species in fish_species_names:
    fish_species = fish_species.replace('_', '%20')
    url = "http://v3.boldsystems.org/index.php/API_Public/sequence?taxon=" + fish_species
    response = requests.get(url, allow_redirects=True)
    print(fish_species, response.status_code)   
    fish_species = fish_species.replace('%20', '_')
    outfile_name = 'COI_data_boldsystems/' + fish_species + '.fas'
    outfile = open(outfile_name, 'wb')
    outfile.write(response.content)
    outfile.close()

In [ ]:
# 2 make single fasta file and filter out sequences that are not COI-5P
full_fasta_data = ''
line_counter = 0    # add number to allow Clustalw2 to run (restricts sequence identifiers to 30 characters)
COI_tracker = False
for fish_species in fish_species_names:
    file_name = fish_species + '.fas'
    try:
        with open('COI_data_boldsystems/' + file_name) as f:
            for line in f:
                if line[0] == '>':    # new sequence name
                    if 'COI-5P' in line:
                        full_fasta_data += line[0] + str(line_counter) + line[1:]
                        line_counter += 1
                        COI_tracker = True
                    else:
                        COI_tracker = False
                else:
                    if COI_tracker:
                        full_fasta_data += line    # new sequence DNA string, add only if COI-5P
                        COI_tracker = False
        f.close()
    except:
        print('species ' + fish_species + ' not found')

outfile_full_fasta_data = open('COI_data_boldsystems/full_fasta_data.fas','w')
outfile_full_fasta_data.write(full_fasta_data)
outfile_full_fasta_data.close()
print (full_fasta_data)    # this yields a total of 2746 sequences on 4 Oct 2018

In [ ]:
import sys
!{sys.executable} -m pip install biopython

In [ ]:
# quick check that sequences look ok
from Bio import SeqIO
for seq_record in SeqIO.parse('COI_data_boldsystems/full_fasta_data.fas', 'fasta'):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

In [ ]:
# produce command line for Clustalw2
from Bio.Align.Applications import ClustalwCommandline
cline = ClustalwCommandline("clustalw2", infile='full_fasta_data.fas')
print(cline)    #run the line printed below in terminal, make sure you have Clustalw2 installed and it is in your path
# on Linux it is possible to run the command line from Jupyter directly, as follows:
#![command line]

In [ ]:
from Bio import AlignIO
align = AlignIO.read("COI_data_boldsystems/mini_fasta_data.aln", "clustal")
print(align)